In [ ]:
from google.colab import drive
import sys
drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/AIM')
!ls -l '/content/gdrive/MyDrive/AIM'
import aim_util

Mounted at /content/gdrive
total 48574
-rw------- 1 root root    1108 May  2 18:51 aim_util.py
-rw------- 1 root root    2696 Apr 15 14:07 citizen_1864_20240414_213308040_groundtruth.txt
-rw------- 1 root root 3663644 Apr 14 21:34 citizen_1864_20240414_213308040.jpg
-rw------- 1 root root    2696 May  2 20:40 citizen_1864_groundtruth_annotated.txt
-rw------- 1 root root    3899 May  3 15:25 citizen_1864_groundtruth_word_list_lower.txt
-rw------- 1 root root    3899 May  3 15:25 citizen_1864_groundtruth_word_list.txt
-rw------- 1 root root    3899 May  3 15:25 citizen_1864_groundtruth_word_string_lower.txt
-rw------- 1 root root    3899 May  3 15:25 citizen_1864_groundtruth_word_string.txt
-rw------- 1 root root 3663644 Apr 23 20:54 citizen_1864_rescan.jpg
-rw------- 1 root root   68979 May  3 18:44 citizen_1864_xform0_carlv_bbox.txt
-rw------- 1 root root    3778 May  3 18:44 citizen_1864_xform0_carlv_word_list.txt
-rw------- 1 root root    2425 May  3 18:44 citizen_1864_xform0_carlv_w

In [ ]:
!ls -l /content/gdrive/MyDrive/AIM/*_word_list.txt

-rw------- 1 root root 3899 May  3 15:25 /content/gdrive/MyDrive/AIM/citizen_1864_groundtruth_word_list.txt
-rw------- 1 root root 3778 May  3 15:23 /content/gdrive/MyDrive/AIM/citizen_1864_xform0_carlv_word_list.txt
-rw------- 1 root root 3598 May  2 20:37 /content/gdrive/MyDrive/AIM/citizen_1864_xform1_carlv_word_list.txt
-rw------- 1 root root 3394 May  2 20:40 /content/gdrive/MyDrive/AIM/citizen_1864_xform2_carlv_word_list.txt
-rw------- 1 root root 3532 May  2 20:44 /content/gdrive/MyDrive/AIM/citizen_1864_xform3_carlv_word_list.txt
-rw------- 1 root root 3621 May  2 20:47 /content/gdrive/MyDrive/AIM/citizen_1864_xform4_carlv_word_list.txt


In [ ]:
import ast

groundtruth_path = '/content/gdrive/MyDrive/AIM/citizen_1864_groundtruth_word_list_lower.txt'
pred_xform0_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform0_carlv_word_list.txt'
pred_xform1_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform1_carlv_word_list.txt'
pred_xform2_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform2_carlv_word_list.txt'
pred_xform3_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform3_carlv_word_list.txt'
pred_xform4_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform4_carlv_word_list.txt'

def file_to_list(path):
    with open(path, 'r') as f:
        target_list = ast.literal_eval(f.read())
    return target_list

groundtruth_list = file_to_list(groundtruth_path)
pred_xform0_list = file_to_list(pred_xform0_path)
pred_xform1_list = file_to_list(pred_xform1_path)
pred_xform2_list = file_to_list(pred_xform2_path)
pred_xform3_list = file_to_list(pred_xform3_path)
pred_xform4_list = file_to_list(pred_xform4_path)

In [ ]:
!pip install fuzzywuzzy

In [ ]:
# Model A
import fuzzywuzzy.fuzz

# groundtruth_list = ['sun', 'car', 'bed', 'fox', 'ate', 'drum', 'noon', 'city', 'said', 'said', 'said', 'said', 'gun', 'zoo', 'joy', 'prizes']
# prediction_list = ['sun', 'car', 'bed', 'fox', 'ate', 'drum', 'noontime', 'city', 'prizes', 'said', 'said', 'said', 'said', 'hit', 'prize', 'princes']

#prediction_list = pred_xform0_list

def rate_match(groundtruth_list, prediction_list):
  print(f"groundtruth_list =>{groundtruth_list}")
  print(f"prediction list=>{prediction_list}")

  # Partial match threshold (experiment with this value)
  PARTIAL_MATCH_THRESHOLD = 80

  # Calculate lengths
  groundtruth_length = len(groundtruth_list)
  prediction_length = len(prediction_list)

  # Find matches, partial matches, and mismatches
  matches = []
  partial_matches = []
  mismatches = []

  used_groundtruth = set()  # Track which groundtruth words have been matched

  for word in prediction_list:
      best_match_found = False
      ###################!!!!!!!!!!!##################
      # TODO: multiple occurances of word are excluded!
      for groundtruth_word in groundtruth_list:
          if groundtruth_word in used_groundtruth:
              continue  # Skip already matched groundtruth words

          ratio = fuzzywuzzy.fuzz.ratio(word, groundtruth_word)
          if ratio == 100:
              matches.append(word)
              used_groundtruth.add(groundtruth_word)  # Mark as matched
              best_match_found = True
              break  # Move on once an exact match is found

      if not best_match_found:
          # No exact match, now check for partial matches
          for groundtruth_word in groundtruth_list:
              if groundtruth_word in used_groundtruth:
                  continue

              ratio = fuzzywuzzy.fuzz.ratio(word, groundtruth_word)
              if ratio >= PARTIAL_MATCH_THRESHOLD:
                  partial_matches.append(word)
                  break  # A partial match is enough

      if not best_match_found and word not in partial_matches:
          mismatches.append(word)

  # Calculate percentages
  num_matches = len(matches)
  num_partial_matches = len(partial_matches)
  accuracy_percentage = (num_matches / prediction_length) * 100
  partial_match_percentage = (num_partial_matches / prediction_length) * 100

  # Print the results
  print("Groundtruth list length:", groundtruth_length)
  print("Prediction list length:", prediction_length)
  print("Overall accuracy:", accuracy_percentage, "%")
  print("Partial match percentage:", partial_match_percentage, "%")
  print("\nMatching words:")
  print(matches)
  print("\nPartially Matching words:")
  print(partial_matches)
  print("\nMismatched words:")
  print(mismatches)

  return num_matches, num_partial_matches, accuracy_percentage, partial_match_percentage

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
print(f"==========xform0===========")
rate_match(groundtruth_list, pred_xform0_list)
print(f"==========xform1===========")
rate_match(groundtruth_list, pred_xform1_list)
print(f"==========xform2===========")
rate_match(groundtruth_list, pred_xform2_list)
print(f"==========xform3===========")
rate_match(groundtruth_list, pred_xform3_list)
print(f"==========xform4===========")
rate_match(groundtruth_list, pred_xform4_list)

==========xform0===========
groundtruth_list =>['united', 'states', 'of', 'america', 'our', 'liberty', 'we', 'prize', 'and', 'out', 'rights', 'we', 'will', 'maintain', 'state', 'of', 'iowa', 'ss:', 'scott', 'county', 'be', 'it', 'remembered', 'that', 'at', 'a', 'term', 'of', 'the', 'district', 'court', 'holden', 'in', 'and', 'for', 'said', 'county', 'in', 'the', 'city', 'of', 'davenport', 'therein', 'on', 'the', '7', 'day', 'of', 'november', 'in', 'the', 'year', 'of', 'our', 'lord', 'one', 'thousand', 'eight', 'hundred', 'sixtyfour', 'was', 'presentthe', 'honorable', 'j.', 'scott', 'richman', 'sole', 'presiding', 'judge', 'john', 'm.', 'lyter', 'sheriff', 'of', 'said', 'county', 'and', 'ira', 'm.', 'gifford', 'clerk', 'of', 'said', 'court', 'when', 'the', 'fol-', 'lowing', 'among', 'other', 'proceedings', 'were', 'had', 'to', 'wit:', 'hans', 'claus', 'olson', 'a', 'native', 'of', 'norway', 'and', 'at', 'present', 'residing', 'within', 'said', 'state', 'appeared', 'in', 'open', 'court',

(130, 84, 30.73286052009456, 19.858156028368796)

In [ ]:
!pip3 install git+https://github.com/casics/nostril.git

  Cloning https://github.com/casics/nostril.git to /tmp/pip-req-build-vbsac6p5
  Running command git clone --filter=blob:none --quiet https://github.com/casics/nostril.git /tmp/pip-req-build-vbsac6p5
  Resolved https://github.com/casics/nostril.git to commit fbc0c91249283a9fbc9036206391ce1138826fd3
  Preparing metadata (setup.py) ... done
  Created wheel for nostril: filename=nostril-1.2.0-py3-none-any.whl size=5765786 sha256=00320cee9430b3b1ebe9cea3ba78b7cfb654a1f8301e307fbe10862afe9a0487
  Stored in directory: /tmp/pip-ephem-wheel-cache-8g0kl5fv/wheels/de/3e/43/5b766704a7dbffce33fcbd15a63a9919cb9cc743e04780b9d6
Successfully built nostril


In [ ]:
pred_list = ['united', 'states', 'of', 'america', 'luere', 'ralling', 'o', 'o', 'v', 'state', 'of', 'iowa', 'sss', 'co', 'scott', 'cunty', 'be', 'it', 'renteutbeted', 'that', 'at', 'e', 'term', 'or', 'the', 'district', 'court', 'holden', 'n', 'and', 'for', 'said', 'county', 'n', 'the', 'e', 'ovcm', 'elc', 'city', 'of', 'davenport', 'theein', 'on', 'the', 'day', 'of', 'in', 'the', 'year', 'of', 'our', 'lord', 'one', 'o', 'susty', 'thousand', 'eight', 'hundre', 'and', 'fuw', 'was', 'present', 'the', 'hororable', 'je', 'scott', 'richmany', 'sole', 'prsiding', 'judge', 'john', 'me', 'lyter', 'sheriff', 'of', 'said', 'county', 'and', 'ira', 't', 'gifford', 'clerk', 'of', 'said', 'court', 'whenthe', 'fol', 'v', 'anws', 'cans', 'olen', 'lowing', 'among', 'other', 'roceedings', 'were', 'had', 'towits', 's', 'a', 'native', 'of', 'a', 'loc', 'prcsens', 'resian', 'vltmm', 'said', 'stater', 'appeared', 'lile', 'open', 'cout', 'icctolente', 'be', 'aemitted', 'oeceme', 'v', 'a', 'citizen', 'of', 'thie', 'uited', 'states', 'oww', 'and', 'it', 'roadwo', 'appearing', 'to', 'cot', 'the', 'oo', 'satifaction', 'of', 'ssked', 'the', 'court', 'that', 'oouns', 'he', 'had', 'declaredn', 'oath', 'c', 'before', 'e', 'a', 'court', 'gf', 'record', 'heing', 'common', 'law', 'jurisdiction', 'and', 'using', 'cl', 'sal', 'two', 'years', 'at', 'least', 'before', 'his', 'admissions', 'that', 'it', 'e', 'was', 'bona', 'fide', 'his', 'inter', 'tion', 'to', 'become', 'a', 'citizen', 'of', 'the', 'united', 'states', 'and', 'to', 'renounce', 'forever', 'all', 'allegiance', 'to', 'cas', 'o', 'foreign', 'prince', 'potentater', 'state', 'or', 'sovereigntys', 'whatsoever', 'and', 'particularly', 'to', 'iw', 'wedens', 'o', 'subile', 'of', 'wh', 'he', 'was', 'heretofore', 'a', 'and', 'laid', 'applicant', 'having', 'declared', 'on', 'oath', 'before', 'this', 'co', 'court', 'that', 'he', 'will', 'support', 'the', 'constitution', 'of', 'the', 'united', 'statess', 'nd', 'that', 'he', 'doth', 'absolutely', 'and', 'entirely', 'renounce', 's', 'and', 'abjure', 'all', 'allegiance', 'and', 'fidelity', 'to', 'every', 'foreign', 'princer', 'poentater', 'state', 'or', 'sovereigntys', 'whatsoever', 'nd', 'pars', 'ticularly', 'to', 'the', 'power', 'above', 'named', 'the', 'court', 'being', 'satisfied', 'that', 'said', 'applicant', 'has', 'resided', 'within', 'the', 'united', 'o', 'states', 'for', 'the', 'term', 'offive', 'years', 'next', 'preceeding', 'his', 'admission', 'without', 'being', 'at', 'any', 'time', 'during', 'the', 'said', 'fre', 'years', 'out', 'of', 'the', 'territory', 'of', 'the', 'united', 'states', 'and', 'within', 'this', 'state', 'onc', 'year', 'at', 'leasty', 'and', 'it', 'further', 'appearing', 'to', 'the', 'n', 'n', 'satisfaction', 'of', 'this', 'court', 'that', 'during', 'that', 'time', 'he', 'has', 'behaved', 'as', 'a', 'man', 'of', 'good', 'moral', 'character', 'attached', 'to', 'the', 'n', 'constitution', 'of', 'the', 'united', 'states', 'and', 'well', 'disposed', 'to', 'the', 'good', 'order', 'and', 'happiness', 'of', 'the', 'same', 'thereuon', 'the', 'court', 'admitted', 'the', 'sald', 'applicant', 'to', 'become', 'a', 'citizen', 'of', 'tho', 'unitod', 'states', 'and', 'ordered', 'all', 'proceedings', 'aoresaid', 'eo', 'to', 'e', 'be', 'entere', 'dl', 'eeo', 'llic', 'cout', 's', 'y', 'testimotty', 'aleeon', 'l', 'ira', 'm', 'giffordy', 'olerk', 'of', 'the', 'court', 'aforesaul', 'have', 'si', 'hereto', 'set', 'my', 'hand', 'and', 'aflxed', 'the', 'seal', 'of', 'sauid', 'court', 'at', 'ofice', 'in', 'hes', 'city', 'e', 'of', 'dawenport', 'in', 'said', 'county', 'this', 'the', 'seen', 'day', 's', 'nnem', 'bes', 'in', 'the', 'year', 'of', 'ourlord', 'one', 'thousanl', 'eight', 'hundred', 'and', 'sixty', 'sangezc', 'clerk', 'f', 'district', 'court', 'n']


In [ ]:
from nostril import nonsense
keyword_list = []
gibberish_list = []

for s in pred_list:
    if nonsense(s):
        gibberish_list.append(s)
    else:
        keyword_list.append(s)

print("Keyword list:", keyword_list)
print("Gibberish list:", gibberish_list)

ValueError: Text is too short to test

In [ ]:
!pip install random_string_detector

In [ ]:
from random_string_detector import RandomStringDetector

detector = RandomStringDetector()

keyword_list = []
gibberish_list = []

for s in pred_list:
    if detector(s) or len(s)<2:
        gibberish_list.append(s)
    else:
        keyword_list.append(s)

print("Keyword list:", keyword_list)
print("Gibberish list:", gibberish_list)

Keyword list: ['united', 'states', 'of', 'america', 'luere', 'ralling', 'state', 'of', 'iowa', 'sss', 'co', 'scott', 'cunty', 'be', 'it', 'renteutbeted', 'that', 'at', 'term', 'or', 'the', 'district', 'court', 'holden', 'and', 'for', 'said', 'county', 'the', 'elc', 'city', 'of', 'davenport', 'theein', 'on', 'the', 'day', 'of', 'in', 'the', 'year', 'of', 'our', 'lord', 'one', 'susty', 'thousand', 'eight', 'hundre', 'and', 'fuw', 'was', 'present', 'the', 'hororable', 'je', 'scott', 'richmany', 'sole', 'prsiding', 'judge', 'john', 'me', 'lyter', 'sheriff', 'of', 'said', 'county', 'and', 'ira', 'gifford', 'clerk', 'of', 'said', 'court', 'whenthe', 'fol', 'anws', 'cans', 'olen', 'lowing', 'among', 'other', 'roceedings', 'were', 'had', 'towits', 'native', 'of', 'loc', 'prcsens', 'resian', 'vltmm', 'said', 'stater', 'appeared', 'lile', 'open', 'cout', 'icctolente', 'be', 'aemitted', 'oeceme', 'citizen', 'of', 'thie', 'uited', 'states', 'oww', 'and', 'it', 'roadwo', 'appearing', 'to', 'cot', '

In [ ]:
print(f"==========unfiltered===========")
rate_match(groundtruth_list, pred_list)

print(f"==========no nonsense===========")
rate_match(groundtruth_list, keyword_list)

==========unfiltered===========
groundtruth_list =>['united', 'states', 'of', 'america', 'our', 'liberty', 'we', 'prize', 'and', 'out', 'rights', 'we', 'will', 'maintain', 'state', 'of', 'iowa', 'ss:', 'scott', 'county', 'be', 'it', 'remembered', 'that', 'at', 'a', 'term', 'of', 'the', 'district', 'court', 'holden', 'in', 'and', 'for', 'said', 'county', 'in', 'the', 'city', 'of', 'davenport', 'therein', 'on', 'the', '7', 'day', 'of', 'november', 'in', 'the', 'year', 'of', 'our', 'lord', 'one', 'thousand', 'eight', 'hundred', 'sixtyfour', 'was', 'presentthe', 'honorable', 'j.', 'scott', 'richman', 'sole', 'presiding', 'judge', 'john', 'm.', 'lyter', 'sheriff', 'of', 'said', 'county', 'and', 'ira', 'm.', 'gifford', 'clerk', 'of', 'said', 'court', 'when', 'the', 'fol-', 'lowing', 'among', 'other', 'proceedings', 'were', 'had', 'to', 'wit:', 'hans', 'claus', 'olson', 'a', 'native', 'of', 'norway', 'and', 'at', 'present', 'residing', 'within', 'said', 'state', 'appeared', 'in', 'open', 'cou

(131, 79, 31.873479318734795, 19.22141119221411)